In [ ]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sn
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import random
import os
import cv2
from skimage.io import imread
from tensorflow import keras
import tensorflow as tf

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense, Activation, BatchNormalization


In [ ]:
# Definimos el tamaño de la imagen (125x125) y los paths
IMAGE_SIZE = 125

TRAIN_PATH = "./data/movida/train/"

TEST_PATH = "./data/movida/test/"

In [ ]:
# ------------------------------------------df_test---------------------------------------------

def read_data(path):
    archivos = []
    for file in os.listdir(path): 
            archivos.append(file)            
    return np.array(archivos)
    
archivos_test = read_data(TEST_PATH)

df_test = pd.DataFrame({
    'archivo': archivos_test})


# -----------------------------------------df_train-----------------------------------------------

df_train = pd.read_csv("./data/movida/file_target.csv")
  

In [ ]:
print('train',df_train.head())
print('train shape',df_train.shape)
print('test',df_test.head())
print('test shape',df_test.shape)

In [ ]:
df_train.info()

In [ ]:
def read_data2(df, path, image_size):
    X = []
    Y = []
    
    for index, row in df.iterrows():
        image = imread(os.path.join(path, row['archivo']))
        
        smallimage = cv2.resize(image, (IMAGE_SIZE,IMAGE_SIZE))
        X.append(smallimage)

    return np.array(X)

X_test = read_data2(df_test, TEST_PATH, IMAGE_SIZE)

X_test_norma = X_test / 255.0

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

# Add our data-augmentation parameters to ImageDataGenerator
train_datagen = ImageDataGenerator(rescale = 1./255.,                                   
                                   rotation_range = 40,
                                   width_shift_range = 0.2,
                                   height_shift_range = 0.2,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

# Note that the validation data should not be augmented!
validation_datagen = ImageDataGenerator(rescale = 1.0/255. )

In [ ]:
train_generator = train_datagen.flow_from_dataframe(df_train,
                                                    TRAIN_PATH,
                                                    x_col='archivo',
                                                    y_col='target',
                                                    batch_size = 20,
                                                    class_mode = 'sparse',
                                                    target_size = (IMAGE_SIZE, IMAGE_SIZE))


In [ ]:
## VGG16

from tensorflow.keras.applications.vgg16 import VGG16

base_model = VGG16(input_shape = (IMAGE_SIZE, IMAGE_SIZE, 3),
                  include_top=False)

for layer in base_model.layers:
    layer.trainable = False

    
##### FULLY CONNECTED LAYER #####
# Flatten the output layer to 1 dimension
x = keras.layers.Flatten()(base_model.output)

# Add a fully connected layer with 512 hidden units and ReLU activation
x = keras.layers.Dense(1000, activation='relu')(x)

# Add a fully connected layer with 512 hidden units and sigmoid activation
x = keras.layers.Dense(1000, activation='sigmoid')(x)

# Add a fully connected layer with 512 hidden units and ReLU activation
x = keras.layers.Dense(1000, activation='relu')(x)

# Add a dropout rate of 0.5
x = keras.layers.Dropout(0.5)(x)

# Add a final sigmoid layer for classification
x = keras.layers.Dense(29, activation='softmax')(x)

model = tf.keras.models.Model(base_model.input, x)

model.compile(optimizer = 'adam', loss = 'sparse_categorical_crossentropy',metrics = ['acc'])

In [ ]:
vgghist = model.fit(train_generator,
                    epochs = 35)

In [ ]:
predictions = model.predict(X_test_norma)

In [ ]:
## EfficientNetB7

import tensorflow.keras.applications as apli
from tensorflow.keras.optimizers import RMSprop

base_model = apli.EfficientNetB7(input_shape = (IMAGE_SIZE, IMAGE_SIZE, 3),
                                 include_top=True,
                                 weights=None)


base_model.compile(RMSprop(lr=0.0001, decay=1e-6), loss = 'sparse_categorical_crossentropy',metrics = ['acc'])

In [ ]:
efihist = base_model.fit(train_generator, steps_per_epoch = 100,
                    epochs = 100)

In [ ]:
predictions = base_model.predict(X_test_norma)

In [ ]:
model.evaluate(X_test_norma)

In [ ]:
predictions = model.predict(X_test_norma)

In [ ]:
predict = predictions.argmax(axis=1)

predict

In [ ]:
import collections

collections.Counter(predict)

In [ ]:
df_test['target']=predict

In [ ]:
series = {}

for i in range(0,29):
    A = df_train[df_train['target_2']==i]['target'].head(1)
    B =A.values
    series[B[0]]=i

In [ ]:
list(series.keys())[list(series.values()).index(0)]

In [ ]:
df_test_2 = df_test

In [ ]:
def change_number(x):

    return list(series.keys())[list(series.values()).index(x)]

df_test_2['target'] = df_test_2['target'].apply(change_number)


In [ ]:
df_test_2

In [ ]:
df_test_2.to_csv('data/submission.csv', index=False)